class, orientation의 비율에 맞춰 train, val, test로 전체 dataset을 slpit

In [27]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import numpy as np
import seaborn as sns
import os
import shutil
import re
from sklearn.model_selection import train_test_split

In [28]:
# todo
img_dir = '/opt/ml/final_project/data/img/'
ann_dir = '/opt/ml/final_project/data/ann/'

In [29]:
fileids = []
class_num = []
orient = []
img_fileids = []

for idx, img_fileid in enumerate(os.listdir(img_dir)):
    
    fileid = re.match('\w+_\d+', img_fileid).group(0)
    ann_fileid = fileid+'.json'

    img_fileids.append(img_fileid)
    fileids.append(fileid)
    with open(ann_dir+ann_fileid, "r", encoding="utf8") as f:
        contents = f.read() # string 타입
        json_data = json.loads(contents)
        class_num.append(json_data['objects'][0]['classTitle'])
        
        tags = json_data['objects'][0]['tags']
        for idx, tag in enumerate(tags):
            if tag['name'] == 'orientation':
                orient.append(tag['value'])

In [ ]:
df = pd.DataFrame()
df['fileid'] = fileids
df['img'] = img_fileids
df['class_num'] = class_num
df['orient'] = orient
df

In [ ]:
df.groupby(['class_num', 'orient']).count()

In [7]:
df_0 = df.loc[df['class_num']=='0']
df_1 = df.loc[df['class_num']=='1']
df_2 = df.loc[df['class_num']=='2']
df_3 = df.loc[df['class_num']=='3']

train,val,test dataset 비율에 따라 test_size 변경

In [10]:
data_0 = df_0[['img', 'fileid']]
target_0 = df_0['orient']
data_0, data_0_val, target_0, target_0_val = train_test_split(data_0, target_0, test_size=0.18, shuffle=True, stratify=target_0, random_state=1)
data_0_train, data_0_test, target_0_train, target_0_test = train_test_split(data_0, target_0, test_size=0.11, shuffle=True, stratify=target_0, random_state=1)

data_1 = df_1[['img', 'fileid']]
target_1 = df_1['orient']
data_1, data_1_val, target_1, target_1_val = train_test_split(data_1, target_1, test_size=0.14, shuffle=True, stratify=target_1, random_state=1)
data_1_train, data_1_test, target_1_train, target_1_test = train_test_split(data_1, target_1, test_size=0.08, shuffle=True, stratify=target_1, random_state=1)

data_2 = df_2[['img', 'fileid']]
target_2 = df_2['orient']
data_2, data_2_val, target_2, target_2_val = train_test_split(data_2, target_2, test_size=0.35, shuffle=True, stratify=target_2, random_state=1)
data_2_train, data_2_test, target_2_train, target_2_test = train_test_split(data_2, target_2, test_size=0.28, shuffle=True, stratify=target_2, random_state=1)

data_3 = df_3[['img', 'fileid']]
target_3 = df_3['orient']
data_3, data_3_val, target_3, target_3_val = train_test_split(data_3, target_3, test_size=0.25, shuffle=True, stratify=target_3, random_state=1)
data_3_train, data_3_test, target_3_train, target_3_test = train_test_split(data_3, target_3, test_size=0.17, shuffle=True, stratify=target_3, random_state=1)

In [12]:
data_train = pd.concat([data_0_train, data_1_train, data_2_train, data_3_train])
data_val = pd.concat([data_0_val, data_1_val, data_2_val, data_3_val])
data_test = pd.concat([data_0_test, data_1_test, data_2_test, data_3_test])

In [14]:
for idx in data_train.index:
    img_path = data_train['img'][idx]
    ann_path = data_train['fileid'][idx]+'.json'
    
    img_origin = img_dir+img_path
    ann_origin = ann_dir+ann_path
    img_copy = '/opt/ml/final_project/data/train/img/'+img_path
    ann_copy = '/opt/ml/final_project/data/train/ann/'+ann_path
    shutil.copy(img_origin, img_copy)
    shutil.copy(ann_origin, ann_copy)

In [18]:
for idx in data_val.index:
    img_path = data_val['img'][idx]
    ann_path = data_val['fileid'][idx]+'.json'
    
    img_origin = img_dir+img_path
    ann_origin = ann_dir+ann_path
    img_copy = '/opt/ml/final_project/data/val/img/'+img_path
    ann_copy = '/opt/ml/final_project/data/val/ann/'+ann_path
    shutil.copy(img_origin, img_copy)
    shutil.copy(ann_origin, ann_copy)

In [20]:
for idx in data_test.index:
    img_path = data_test['img'][idx]
    ann_path = data_test['fileid'][idx]+'.json'
    
    img_origin = img_dir+img_path
    ann_origin = ann_dir+ann_path
    img_copy = '/opt/ml/final_project/data/test/img/'+img_path
    ann_copy = '/opt/ml/final_project/data/test/ann/'+ann_path
    shutil.copy(img_origin, img_copy)
    shutil.copy(ann_origin, ann_copy)

In [35]:
df['orient']=='Rear'

0       False
1        True
2       False
3       False
4       False
        ...  
1122    False
1123    False
1124    False
1125    False
1126    False
Name: orient, Length: 1127, dtype: bool

In [38]:
df.loc[(df['orient']=='Rear') & (df['class_num']=='2')]

,fileid,class_num,orient
129,self_0129,2,Rear
134,self_0134,2,Rear


In [31]:
df.groupby(['class_num']).count()

,fileid,orient
class_num,,
0,420,420
1,570,570
2,17,17
3,120,120


In [32]:
data = df['Title']
target = df['class_num']
x, x_test, y, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=1)

In [34]:
print(data.shape)
print(target.shape)
print(x.shape)
print(y.shape)

(1127,)
(1127,)
(901,)
(901,)


In [35]:
x_train, x_val, y_val, y_val = train_test_split(x, y, test_size=0.25, shuffle=True, stratify=y, random_state=1)

In [36]:
x_train

27      self_0027
1060    yt_2_0087
83      self_0083
227     self_0228
335     self_0336
          ...    
910     yt_1_0160
632     self_0633
788     yt_1_0038
460     self_0461
1095    yt_2_0124
Name: Title, Length: 675, dtype: object

In [37]:
x_val

1059    yt_2_0086
457     self_0458
52      self_0052
956     yt_1_0206
339     self_0340
          ...    
823     yt_1_0073
434     self_0435
356     self_0357
876     yt_1_0126
1114    yt_2_0144
Name: Title, Length: 226, dtype: object

In [38]:
x_test

279    self_0280
365    self_0366
563    self_0564
552    self_0553
444    self_0445
         ...    
7      self_0007
677    web_0034.
947    yt_1_0197
181    self_0181
923    yt_1_0173
Name: Title, Length: 226, dtype: object